In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time # for sleep time
import csv
import re
import functools
import datetime #for csv file title
from bs4 import BeautifulSoup as bs
import requests

house_title_list = []

driver = webdriver.Chrome(r'/Users/mohamedsellamia/airbnb/chromedriver')

driver.get('https://www.airbnb.com/s/Toronto--ON--Canada/homes?refinement_paths%5B%5D=%2Fhomes&current_tab_id=home_tab&selected_tab_id=home_tab&search_type=filter_change&adults=1&place_id=ChIJpTvG15DL1IkRd8S0KlBVNTI&screen_size=large&hide_dates_and_guests_filters=false')

now = datetime.datetime.now() #for csv file title

#For csv file names
def reduce_concat(x, sep=""):
    return functools.reduce(lambda x, y: str(x) + sep + str(y), x)

def paste(*lists, sep=" ", collapse=None):
    result = map(lambda x: reduce_concat(x, sep=sep), zip(*lists))
    if collapse is not None:
        return reduce_concat(result, sep=collapse)
    return list(result)

#Open CSV Writer
filename = paste(['airBnb_To'], [now.year], ['-'], [now.month], ['-'], [now.day],['.csv'], sep='')
filename = ''.join(filename)
csv_file = open(filename, 'w', encoding='utf-8',newline = '')
writer = csv.writer(csv_file)

index = 1

house_all_links = []

while True:
    
    if index > 17 :
        break
    try:
        print("Getting list of links from Page " + str(index))
        index = index + 1
        #driver.maximize_window()
        time.sleep(6) # give new page time to load
                
        house_list = (driver.find_elements_by_xpath('//a[@class="_1wtrbgoo"]'))

        house_links = [house.get_attribute('href') for house in house_list ] 
        
        house_all_links.extend(house_links)
  
        # Locate the next button on the page.
        wait_button = WebDriverWait(driver, 10)
        
        next_button = wait_button.until(EC.element_to_be_clickable((By.XPATH,'//li[@class="_r4n1gzb"]')))
        
        next_button.click()

        
    except Exception as e:
        print(e)
        continue
        
print(house_all_links)
for link in house_all_links: 
    page = requests.get(link)
    soup = bs(page.text, "lxml")
    match = soup.find_all('a', href=True)
    link_list = []
    for i in match:
        link = i["href"]
        link_list.append(link)  
    for url in link_list[2:20]:
        url = ("https://www.airbnb.ca"+str(url))
        airbnb_dict = {}
        driver.get(url)
        time.sleep(7)

        try:
            house_title = driver.find_element_by_xpath('//h1/span[@class="_18hrqvin"]').text
        except:
            house_title = None
        try:
            house_type =  driver.find_element_by_xpath('//div[@class="_504dcb"]').text
        except:
            house_type = None
        try:
            house_location =  driver.find_element_by_xpath('//div[@class="_czm8crp"]').text
        except:
            house_location = None

        try:
            price = driver.find_element_by_xpath('//span/span[@class="_doc79r"]').text
        except:
            price = None        

        try:
            no_of_reviews = driver.find_element_by_xpath('//div[@class="_14hveccc"]').text
        except:
            no_of_reviews = None  

        try:
            rating = driver.find_element_by_xpath('//div[@class="_10za72m2"]').text

        except:
            rating = None

        try:
            max_occupancy = driver.find_elements_by_xpath('//div/div[@class="_36rlri"]/div[@class="_czm8crp"]')
            house_detail = [detail.text for detail in max_occupancy ]

        except:

            house_detail = None
            
        try:
            amenities = driver.find_elements_by_xpath('//*[@id="amenities"]//div[@class ="_czm8crp"]')
            amenity_detail = [amenity.text for amenity in amenities]

        except:

            house_detail = None
        if house_title not in house_title_list:
            house_title_list.append(house_title)
            print ([house_title, house_type, house_location, price, no_of_reviews, rating, house_detail,amenity_detail])
            
            airbnb_dict['house_title'] = house_title
            airbnb_dict['house_type'] = house_type
            airbnb_dict['house_locatione'] = house_location
            airbnb_dict['price'] = price
            airbnb_dict['no_of_reviews'] = no_of_reviews
            airbnb_dict['rating'] = rating
            airbnb_dict['house_detail'] = house_detail
            airbnb_dict['amenity_detail'] = amenity_detail
            writer.writerow(airbnb_dict.values())

csv_file.close()
driver.close()


Getting list of links from Page 1
Message: element click intercepted: Element <li class="_r4n1gzb">...</li> is not clickable at point (529, 651). Other element would receive the click: <div class="optanon-alert-box-wrapper  " role="alertdialog" aria-labelledby="alert-box-title" aria-describedby="alert-box-message" style="display: block; animation-name: slide-down-custom; animation-duration: 1000ms; animation-fill-mode: forwards; animation-timing-function: ease-out;">...</div>
  (Session info: chrome=78.0.3904.70)

Getting list of links from Page 2
Getting list of links from Page 3
Getting list of links from Page 4
Getting list of links from Page 5
Getting list of links from Page 6
Getting list of links from Page 7
Getting list of links from Page 8
Getting list of links from Page 9
Getting list of links from Page 10
Getting list of links from Page 11
Getting list of links from Page 12
Getting list of links from Page 13
Getting list of links from Page 14
Getting list of links from Page 1

['Cozy 1 bedroom apartment (2 beds) close to subway.', '\U000f1001\nEntire home\nYou’ll have the apartment to yourself.', 'Toronto', '$55', '508 reviews', '4.79', ['2 guests', '1 bedroom', '2 beds', '1 bath', '\U000f0004\n508 Reviews', '\U000f0019\nVerified'], ['Kitchen', 'Laptop-friendly workspace', 'TV', 'Iron', 'Unavailable: Carbon monoxide detector\nCarbon monoxide detector']]
['Studio Yellow at Downtown Core', '\U000f1001\nEntire home\nYou’ll have the serviced apartment to yourself.', 'Toronto', '$71', '380 reviews', '4.46', ['4 guests', 'Studio', '2 beds', '1 bath', '\U000f0004\n7997 Reviews', '\U000f0019\nVerified'], ['Elevator', 'Wifi', 'Kitchen', 'Laptop-friendly workspace']]
['High Floor | Balcony | Fast WiFi | ❤️ of Downtown', '\U000f1001\nEntire home\nYou’ll have the condominium to yourself.', 'Toronto', '$100', '316 reviews', '4.85', ['2 guests', 'Studio', '1 bed', '1 bath', '\U000f0004\n480 Reviews', '\U000f0019\nVerified'], ['Elevator', 'Wifi', 'Kitchen', 'Hot tub']]
['J

['Slick 1 bedroom loft at Downtown Central', '\U000f1001\nEntire home\nYou’ll have the apartment to yourself.', 'Toronto', '$70', '356 reviews', '4.32', ['6 guests', '1 bedroom', '3 beds', '1 bath', '\U000f0004\n7997 Reviews', '\U000f0019\nVerified'], ['Wifi', 'Kitchen', 'Laptop-friendly workspace', 'Iron']]
["Luxury Suite in Toronto's Entertainment District", '\U000f1001\nEntire home\nYou’ll have the apartment to yourself.', 'Toronto', '$120', '759 reviews', '4.89', ['3 guests', '1 bedroom', '2 beds', '1 bath', '\U000f0004\n759 Reviews', '\U000f1101\n2 References', '\U000f0019\nVerified'], ['Elevator', 'Wifi', 'Kitchen', 'Cable TV']]
['Modern Condo near CN Tower WITH FREE PARKING', '\U000f1001\nEntire home\nYou’ll have the apartment to yourself.', 'Toronto', '$85', '450 reviews', '4.74', ['4 guests', '1 bedroom', '1 bed', '1 bath', '\U000f0004\n450 Reviews', '\U000f0019\nVerified'], ['Free parking on premises', 'Elevator', 'Wifi', 'Kitchen']]
['Near CN Tower | Luxury Condo| One bedroo

['Cozy Room in a Condo', '\U000f0104\nSelf check-in\nCheck yourself in with the lockbox.', 'Toronto', '$26', '177 reviews', '4.67', ['2 guests', '1 bedroom', '1 bed', '1 shared bath', '\U000f0004\n272 Reviews', '\U000f0019\nVerified'], ['Free parking on premises', 'Elevator', 'Indoor fireplace', 'Wifi', 'Unavailable: Carbon monoxide detector\nCarbon monoxide detector']]
['CUTEST HOUSE AROUND (AND NEWLY RENOVATED!)', '\U000f1001\nEntire home\nYou’ll have the house to yourself.', 'Toronto', '$110', '319 reviews', '4.91', ['2 guests', '1 bedroom', '1 bed', '1 bath', '\U000f0004\n319 Reviews', '\U000f0019\nVerified'], ['Wifi', 'Kitchen', 'Laptop-friendly workspace', 'Iron']]
['Downtown Luxury, Private Bathroom!', '\U000f0022\nSparkling clean\n14 recent guests said this place was sparkling clean.', 'Toronto', '$62', '452 reviews', '4.93', ['2 guests', '1 bedroom', '1 bed', '1 bath', '\U000f0004\n1859 Reviews', '\U000f0019\nVerified'], ['Wifi', 'Laptop-friendly workspace', 'TV', 'Iron']]
['L

['Spacious Private Suite: Parking; 1 block to Subway', '\U000f1001\nEntire home\nYou’ll have the house to yourself.', 'Toronto', '$59', '229 reviews', '4.81', ['2 guests', '1 bedroom', '2 beds', '1 bath', '\U000f0004\n321 Reviews', '\U000f0019\nVerified'], ['Free parking on premises', 'Wifi', 'Kitchen', 'Cable TV']]
['Cozy,Trendy & Private Apartment with Free Parking', '\U000f1001\nEntire home\nYou’ll have the apartment to yourself.', 'Toronto', '$80', '213 reviews', '4.91', ['2 guests', '1 bedroom', '1 bed', '1 bath', '\U000f0004\n213 Reviews', '\U000f0019\nVerified'], ['Free parking on premises', 'Wifi', 'Kitchen', 'Laptop-friendly workspace']]
['Newly Furnished Condo w/Balcony across CN Tower', '\U000f1001\nEntire home\nYou’ll have the condominium to yourself.', 'Toronto', '$125', '376 reviews', '4.94', ['6 guests', '1 bedroom', '3 beds', '1 bath', '\U000f0004\n1053 Reviews', '\U000f0019\nVerified'], ['Elevator', 'Wifi', 'Kitchen', 'Cable TV']]
['DownTown living condo', '\U000f1001\

['!!!Best Location, Luxury, Comfortable!!!', '\U000f1001\nEntire home\nYou’ll have the apartment to yourself.', 'Toronto', '$80', '226 reviews', '4.65', ['4 guests', '1 bedroom', '2 beds', '1 bath', '\U000f0004\n285 Reviews', '\U000f0019\nVerified'], ['Elevator', 'Breakfast', 'Wifi', 'Kitchen']]
['Executive Suite - Entertainment District', '\U000f1001\nEntire home\nYou’ll have the apartment to yourself.', 'Toronto', '$100', '244 reviews', '4.93', ['4 guests', '1 bedroom', '2 beds', '1 bath', '\U000f0004\n361 Reviews', '\U000f0019\nVerified'], ['Elevator', 'Wifi', 'Kitchen', 'Cable TV']]
['SPECTACULAR DOWNTOWN CONDO W/ STUNNING CITY VIEWS', '\U000f1001\nEntire home\nYou’ll have the condominium to yourself.', 'Toronto', '$104', '124 reviews', '4.90', ['5 guests', '1 bedroom', '2 beds', '1 bath', '\U000f0004\n124 Reviews', '\U000f0019\nVerified'], ['Elevator', 'Wifi', 'Kitchen', 'Hot tub']]
['Top Location Condo in Heart of Downtown Toronto', '\U000f1001\nEntire home\nYou’ll have the condo

['Home away from home', '\U000f1001\nEntire home\nYou’ll have the townhouse to yourself.', 'Toronto', '$61', '151 reviews', '4.23', ['4 guests', '1 bedroom', '1 bed', '1 bath', '\U000f0004\n424 Reviews', '\U000f0019\nVerified'], ['Free parking on premises', 'Wifi', 'Kitchen', 'Cable TV', 'Unavailable: Carbon monoxide detector\nCarbon monoxide detector']]
['The Best Of Toronto', '\U000f1001\nEntire home\nYou’ll have the condominium to yourself.', 'Toronto', '$99', '107 reviews', '4.73', ['2 guests', '1 bedroom', '1 bed', '1 bath', '\U000f0004\n1093 Reviews', '\U000f0019\nVerified'], ['Elevator', 'Wifi', 'Kitchen', 'Laptop-friendly workspace']]
['Private Unit, Bathroom, Kitchen Downtown Toronto', '\U000f1001\nEntire home\nYou’ll have the apartment to yourself.', 'Toronto', '$41', '71 reviews', '4.77', ['2 guests', 'Studio', '1 bed', '1 bath', '\U000f0004\n229 Reviews', '\U000f0019\nVerified'], ['Wifi', 'Kitchen', 'TV', 'Iron']]
['Modern hotel room-No cleaning fee!+ Breakfast!', '\U000f01

['Large 2 Bed Condo! Downtown Toronto’s Art District', '\U000f1001\nEntire home\nYou’ll have the condominium to yourself.', 'Toronto', '$76', '227 reviews', '4.56', ['4 guests', '2 bedrooms', '2 beds', '1 bath', '\U000f0004\n661 Reviews', '\U000f0019\nVerified'], ['Free parking on premises', 'Elevator', 'Wifi', 'Kitchen', 'Unavailable: Carbon monoxide detector\nCarbon monoxide detector']]
['Comfortable Luxury Condo in the Best Location!', '\U000f1001\nEntire home\nYou’ll have the condominium to yourself.', 'Toronto', '$159', '384 reviews', '4.67', ['3 guests', '1 bedroom', '1 bed', '1 bath', '\U000f0004\n385 Reviews', '\U000f0019\nVerified'], ['Elevator', 'Wifi', 'Kitchen', 'Cable TV']]
['Penthouse 67 fl + FREE Parking', '\U000f1001\nEntire home\nYou’ll have the apartment to yourself.', 'Toronto', '$159', '367 reviews', '4.87', ['3 guests', '1 bedroom', '2 beds', '1 bath', '\U000f0004\n367 Reviews', '\U000f0019\nVerified'], ['Free parking on premises', 'Elevator', 'Wifi', 'Kitchen']]
[

['St Clair West Modern Oasis in Toronto midtown', '\U000f1001\nEntire home\nYou’ll have the guest suite to yourself.', 'Toronto', '$79', '157 reviews', '4.84', ['4 guests', '1 bedroom', '3 beds', '1 bath', '\U000f0004\n189 Reviews', '\U000f1101\n2 References', '\U000f0019\nVerified'], ['Wifi', 'Kitchen', 'Laptop-friendly workspace', 'TV']]
['Luxury Studio + Balcony in the Heart of Downtown', '\U000f1001\nEntire home\nYou’ll have the condominium to yourself.', 'Toronto', '$75', '112 reviews', '4.70', ['2 guests', 'Studio', '1 bed', '1 bath', '\U000f0004\n112 Reviews', '\U000f0019\nVerified'], ['Elevator', 'Wifi', 'Kitchen', 'Gym']]
['Wonderful flat in Leslieville, Toronto', '\U000f1001\nEntire home\nYou’ll have the apartment to yourself.', 'Toronto', '$110', '88 reviews', '4.83', ['2 guests', '1 bedroom', '1 bed', '1 bath', '\U000f0004\n88 Reviews', '\U000f0019\nVerified'], ['Wifi', 'Kitchen', 'Cable TV', 'TV']]
['East Chinatown Bachelor Unit with Paid Parking', '\U000f1001\nEntire home